In [ ]:
#!pip install azure-storage-blob

#mino configuration
curl https://dl.min.io/client/mc/release/linux-amd64/mc \--create-dirs \-o $HOME/minio-binaries/mc
chmod +x $HOME/minio-binaries/mc
export PATH=$PATH:$HOME/minio-binaries/
mc config host ls
mc config host add kubeflow http://minio-service.kubeflow.svc.cluster.local:9000
kubectl -n kubeflow port-forward  --address="0.0.0.0" svc/minio-service 9000:9000
mc ls kubeflow
mc mb kubeflow/kserve-test

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(100, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y[::5, :] += 0.5 - rng.rand(20, 2)

In [ ]:
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_3 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X, y)
regr_2.fit(X, y)
regr_3.fit(X, y)

In [ ]:
# Predict
X_test = np.arange(-100.0, 100.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)
y_3 = regr_3.predict(X_test)

y_3

In [ ]:
import joblib

import os
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minioadmin',
    'AWS_SECRET_ACCESS_KEY': 'minioadmin',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

In [ ]:
bucket_name="kserve-test"
model_path="/home/demo01/model.joblib"
joblib.dump(regr_3, model_path)

In [ ]:
!~/minio-binaries/mc cp ~/model.joblib kubeflow/kserve-test/sklearn-test/model.joblib

In [ ]:
 from kserve import (
    KServeClient,
    utils
)
from kubernetes import client 

In [ ]:
namespace = utils.get_default_target_namespace()
service_name = "sklearn-custom"
KServe = KServeClient()
KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

In [ ]:
isvc_resp

In [ ]:
X_test_list = X_test.tolist()

In [ ]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

In [ ]:
sklearn_test_input={
    "instances": X_test_list
}

In [ ]:
ver_info = '/v1/models/'
operaton = ':predict'
rsvc_url = isvc_url + ver_info + service_name + operaton

response = requests.post(rsvc_url, json=sklearn_test_input)

In [ ]:
response.json()["predictions"] == y_3